In [15]:
import yaml
import pandas as pd
from pathlib import Path
from requests import get
from datetime import datetime
from fhir.resources.observation import Observation
from fhir.resources.reference import Reference
from survey_obs import survey_obs
from fhir.resources.fhirtypesvalidators import ValidationError



In [16]:

'''Dict that maps the column names to the FHIR Observation codes for each LHC-Forms question'''
out_dir = r'/Users/ehaas/Documents/FHIR/SAIG/input/examples-yaml'

In [17]:
def getDisplayText(my_code):
    '''Get the display text for the LOINC code'''
    headers = {'Authorization':"Basic ZWhhYXM6bG9pbmM="}
    url = f'https://fhir.loinc.org/CodeSystem/$lookup?system=http://loinc.org&code={my_code}'
    response = get(url, headers=headers)
    if response.status_code == 200:
        display = next(d.get('valueString') for d in response.json()['parameter'] if d.get('name') == 'display')
        return display
    else:
        return 'Unknown'
# getDisplayText(55757-9)

In [18]:
def write_yaml(fhir_obj):
    out_path = Path(out_dir) / f'Observation-{fhir_obj.id}.yml'
    # note need to the date time format is not json serializabel out of the box see QR-OBs-DE/YAML-JSON-date-serialization.ipynb
    out_path.write_text(fhir_obj.yaml())

In [19]:
'''Using the fhir.resources to create the Observation resource
create panel and items Observations for each row of data in the CSV file
as follows:

1. panel with members that are the items and other panels
'''

def create_observation(row, df, code, category, example_id, items = None):

    my_display = getDisplayText(code.split('[')[0])
    obs = Observation.parse_file("survey_obs.yml")
    obs.id = f"{example_id}-{code.replace('[','answer-').replace(']','')}"
    obs.meta.extension[0].valueString = obs.id.title()
    obs.meta.extension[1].valueMarkdown = f'This is a an example of {code} {my_display} \
for the *US Core Observation Screening Assessment Profile*'
    obs.status = "final"
    obs.category = [{"coding": [{"system": "http://hl7.org/fhir/us/core/CodeSystem/us-core-category", 
                                 "code": category,}]}]
    obs.code = {"coding": [{"system": "http://loinc.org", "code": code, "display": my_display}]}
    obs.subject = {"reference": "Patient/" + df['Respondent_ID'][row]}
    obs.performer = [{"reference": "Performer/" + df['Author'][row]}]
    obs.effectiveDateTime =     mydate = df['Date'][row].isoformat() if "T00:00:00" not in df['Date'][row].isoformat() else df['Date'][row].isoformat()[:-9]
    obs.derivedFrom[0] = {"reference": "DocumentReference/" + df['PDF'][row]}
    #Check if panel
    if items:  # if this is a panel, then no valueCodeableConcep
        obs.hasMember = []
        for i in items:
            ref = Reference()
            ref.reference = f"Observation/{example_id}-{i.replace('[','answer-').replace(']','')}"
            ref.display = getDisplayText(i) if getDisplayText(i) != 'Unknown' else None
            obs.hasMember.append(ref)
            # create the individual Observations for the 3 questions
            # create_observation(df, panel_code=panel_code, item_code=i)
    else: # this is an individual question
        obs.hasMember = None
        value = df[code][row]
        if value =='nan': # then skip this observation
            return
        print(code, row, value)
        if '-' in value: 
            my_display = getDisplayText(value)
            if my_display == 'Unknown':
                obs.valueCodeableConcept = {"coding": [{"system": "http://loinc.org", "code": value}]} 
            else:
                obs.valueCodeableConcept = {"coding": [{"system": "http://loinc.org", "code": value, "display": my_display}]}
        else:
            try:
                obs.valueQuantity = {"value": value}
            except ValidationError as e:
                print(e)
                obs.valueString = value
    write_yaml(obs)

In [20]:
def check_panel_code(lhc_code):
    filter = (df == lhc_code).any()
    sub_df = df.loc[: , filter]
    sub_df

In [21]:
def main():

    df = pd.read_csv("/Users/ehaas/Documents/FHIR/SAIG/input/images/Example3PRAPARE.csv").astype(str)
    df['Date']= pd.to_datetime(df['Date'])
    
    # create the PRAPARE panel
 
    level1 = {
        'code': '93025-5', 'category': 'sdoh', 'itemz': (),
            'multiz': (), 'panelz': ('93043-8','93042-0','93041-2','93040-4','93039-6',),}

    level2 =[{
    'code': '93043-8', 'category': 'sdoh',
    'itemz': ('56051-6', '93035-4', '93034-7', '54899-0',), 
    'multiz': ('32624-9',)
    },
    {'code': '93042-0', 'category': 'sdoh',
    'itemz': ('63512-8','71802-3','93033-9','56799-0',), 
    'multiz': ()
    },
    {'code': '93041-2', 'category': 'sdoh',
    'itemz': ('82589-3','67875-5','76437-3','63586-2',), 
    'multiz': ('93031-3','93030-5',)
    },
    {'code': '93040-4', 'category': 'sdoh',
    'itemz': ('93029-7','93038-8',), 
    'multiz': ()
    },

    {'code': '93039-6', 'category': 'sdoh',
    'itemz': ('93028-9','93027-1','93026-3','76501-6',), 
    'multiz': ()
    },
    ]

    level3 =[
    {'code': '32624-9', 'category': 'sdoh',
    'itemz': ('32624-9[0]','32624-9[1]','32624-9[2]','32624-9[3]','32624-9[4]','32624-9[5]','32624-9[6]','32624-9[7]',)},
    {'code': '93031-3', 'category': 'sdoh',
    'itemz': ('93031-3[0]','93031-3[1]','93031-3[2]','93031-3[3]','93031-3[4]','93031-3[5]','93031-3[6]','93031-3[7]',)},
    {'code': '93030-5', 'category': 'sdoh',
    'itemz': ('93030-5[0]','93030-5[1]','93030-5[2]',)},
    ]

    row = 1
    example_id = 'PRAPARE-Example3'
    #get level 1 panel
    code = level1['code']
    category = level1['category']
    answered_items = [item for item in level1['itemz'] if df[item][row] != 'nan']
    items =  answered_items + list(level1['multiz']) + list(level1['panelz'])
    if items:
        create_observation(row, df, code , category, example_id, items)
    #get level 1 answer codes
    for i in answered_items:
        create_observation(row, df, i, category, example_id)
    #get level 2 panel
    for i in level2:
        code = i['code']
        category = i['category']
        answered_items = [item for item in i['itemz'] if df[item][row] != 'nan']
        items =  answered_items + list(i['multiz'])
        if items:
            create_observation(row, df, code , category, example_id, items)
        #get level 2 answer codes
        for j in answered_items:
            create_observation(row, df, j, category, example_id,) 
    #get level 3 panel
    for i in level3:
        code = i['code']
        category = i['category']
        answered_items = [item for item in i['itemz'] if df[item][row] != 'nan']
        if answered_items:
            create_observation(row, df, code , category, example_id, answered_items)
        #get level 2 answer codes
        for j in answered_items:
            create_observation(row, df, j, category, example_id,) 
        

if __name__ == "__main__":
    main()

56051-6 1 LA32-8
93035-4 1 LA32-8
93034-7 1 LA32-8
54899-0 1 LA43-5
63512-8 1 0
71802-3 1 LA30189-7
93033-9 1 LA32-8
56799-0 1 211 S  JEFFERSON ST, NAPA, CA 94559
1 validation error for Observation
valueQuantity -> value
  value is not a valid decimal (type=type_error.decimal)
82589-3 1 LA30193-9
67875-5 1 LA30136-8
76437-3 1 LA6350-8
63586-2 1 LA30122-8
93029-7 1 LA27722-0
93038-8 1 LA6568-5
32624-9[0] 1 LA30122-8
93030-5[0] 1 LA32-8
